<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Compressive-Transformer" data-toc-modified-id="Compressive-Transformer-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Compressive Transformer</a></span><ul class="toc-item"><li><span><a href="#Options:" data-toc-modified-id="Options:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Options:</a></span></li><li><span><a href="#setup" data-toc-modified-id="setup-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>setup</a></span></li><li><span><a href="#Create-Model" data-toc-modified-id="Create-Model-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Create Model</a></span></li><li><span><a href="#Model-summary" data-toc-modified-id="Model-summary-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Model summary</a></span></li><li><span><a href="#Create-train/test-data-generators" data-toc-modified-id="Create-train/test-data-generators-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Create train/test data-generators</a></span></li><li><span><a href="#Create-addtional-Run-Configurations" data-toc-modified-id="Create-addtional-Run-Configurations-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Create addtional Run Configurations</a></span></li><li><span><a href="#Train" data-toc-modified-id="Train-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Train</a></span></li></ul></li></ul></div>

In [1]:
import os
import re
import sys
import numpy as np
import pandas as pd

from time import sleep, time
from keras import backend as K
from keras.models import Model 
from keras.models import Sequential as SequentialModel
from keras.models import load_model
from keras.layers import Dense, Conv1D, LSTM, Dropout, Embedding, Layer, Input, Flatten, concatenate as Concatenate, Lambda, Add
from keras.callbacks import Callback
import os
import re
import sys
import numpy as np
import pandas as pd

from time import sleep, time
from keras import backend as K
from keras.models import Model 
from keras.models import Sequential as SequentialModel
from keras.models import load_model
from keras.layers import Dense, Conv1D, LSTM, Dropout, Embedding, Layer, Input, Flatten, concatenate as Concatenate, Lambda, Add
from keras.callbacks import Callback
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer as KerasTokenizer
from tqdm.notebook import tqdm
from sometimer import timer, time_this_method

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

sys.path.insert(0, '../ct')

import load
from preprocess import preprocess
from preprocess import Tokenizer
from preprocess.preprocess import separator_samples

from model.layers import LayerNormalization
from model.layers import ContentBasedAttention_CT
from model.layers import ScaledDotProductAttention
from model.layers import MultiHeadAttention
from model.layers import content_based_attention

from model import CompressiveTransformer
from model import AttentionReconstruction

from model.callbacks import ClearCompressedMemory, WriteLogsToFile
from train.generators import next_token_batch_generator
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer as KerasTokenizer
from tqdm.notebook import tqdm
from sometimer import timer, time_this_method

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

sys.path.insert(0, '../ct')

import load
from preprocess import preprocess
from preprocess import Tokenizer
from preprocess.preprocess import separator_samples

from model.layers import LayerNormalization
from model.layers import ContentBasedAttention_CT
from model.layers import ScaledDotProductAttention
from model.layers import MultiHeadAttention
from model.layers import content_based_attention

from model import CompressiveTransformer
from model import AttentionReconstruction

from model.callbacks import ClearCompressedMemory, WriteLogsToFile
from train.generators import next_token_batch_generator

Using TensorFlow backend.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Compressive Transformer

## Options:

In [ ]:
epochs=2
batch_size=128
d_layers=2
sequence_length=128
d_model=128
vocab_size=1024
steps_per_validation=10000

## setup

In [3]:
# Reshape 
def _preprocess(series, batch_size, output_path=None, use_existing=True):
    processed = np.array([ids for english_ids in series for ids in english_ids])
    return processed
    
    
    
#     if use_existing and os.path.exists(output_path):
#         with open(output_path, 'rb') as file:
#             processed = np.load(file)
#         return processed

#     train = [list() for _ in range(batch_size)] 
#     for i, t in enumerate(series):
#         train[i % batch_size].extend(t)

#     sample_length = min(len(t) for t in train)
#     train = [t[:sample_length] for t in train]
#     train = np.array(train)

#     if output_path is not None:
#         with open(output_path, 'wb') as file:
#             np.save(file, train)

#     return train

In [4]:
train_data = pd.read_pickle('../data/wma-en-de/processed/train-en.pkl')
val_data = pd.read_pickle('../data/wma-en-de/processed/val-en.pkl') 

In [5]:
train = _preprocess(train_data.english_ids,
                    batch_size=batch_size,
#                     output_path='../data/wma-en-de/processed/train.np', 
                    use_existing=False)
val = _preprocess(val_data.english_ids, 
                  batch_size=batch_size,
#                   output_path='../data/wma-en-de/processed/val.np', 
                  use_existing=False)

In [6]:
epoch_steps=len(train)  # epoch_steps=train.shape[1]
validation_steps=len(val)  # validation_steps=val.shape[1]
steps_per_epoch=epoch_steps//sequence_length
steps_per_validation= steps_per_validation or validation_steps//sequence_length

## Create Model

In [7]:
ct = CompressiveTransformer(d_layers=d_layers,
                            sequence_length=sequence_length, 
                            d_model=d_model,
                            memory_size=256,
                            compressed_memory_size=256,
                            d_k=16, 
                            d_heads=2, 
                            output_size=vocab_size,
                            batch_size=batch_size,
                            vocab_size=vocab_size)

In [8]:
ct.compile(optimizer='Adam',
           loss='categorical_crossentropy',
           metrics=['accuracy']
          )

<function AttentionReconstruction.attention_reconstruction_loss.<locals>._attention_reconstruction_loss at 0x7f26fc240b00>
   calculating loss...
<function AttentionReconstruction.attention_reconstruction_loss.<locals>._attention_reconstruction_loss at 0x7f26fc24a8c0>
   calculating loss...


## Model summary

In [9]:
ct.summary()

Model: "CompressiveTransformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x (InputLayer)                  (None, 128)          0                                            
__________________________________________________________________________________________________
compressed_memory (InputLayer)  (None, 2, 256, 128)  0                                            
__________________________________________________________________________________________________
memory (InputLayer)             (None, 2, 256, 128)  0                                            
__________________________________________________________________________________________________
h_L0 (Embedding)                (None, 128, 128)     131072      x[0][0]                          
_____________________________________________________________________________

## Create train/test data-generators

In [10]:
generator = next_token_batch_generator(ct=ct,
                                       epochs=epochs, 
                                       data=train,
                                       data_path=None,
                                       epoch_steps=epoch_steps, 
                                       sequence_length=sequence_length, 
                                       batch_size=sequence_length,
                                       stride=1,
                                       vocab_size=vocab_size)
validation_generator = next_token_batch_generator(ct=ct,
                                                  epochs=epochs, 
                                                  data=val,
                                                  data_path=None,
                                                  epoch_steps=validation_steps, 
                                                  sequence_length=sequence_length, 
                                                  batch_size=sequence_length,
                                                  stride=1, 
                                                  vocab_size=vocab_size)

## Create addtional Run Configurations

In [11]:
callbacks = [ClearCompressedMemory(),
             WriteLogsToFile(filepath='training-logs/ct.txt', overwrite_old_file=False)]

## Train

In [ ]:
ct.fit_generator(generator(), 
                 steps_per_epoch=steps_per_epoch,
                 epochs=epochs,
                 callbacks=callbacks,
                 validation_data=validation_generator(),
                 validation_steps=steps_per_validation,
                 shuffle=False
                 )

/home/vs/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
  97111/1626084 [>.............................] - ETA: 128:12:16 - loss: 5.7941 - accuracy: 0.0216

- multiple layers
- larger embedding
- longer dataset
- relative embedding
- attention reconstruction loss